# Bisection Method for Root Finding

## Introduction

The **bisection method** is one of the simplest and most robust numerical techniques for finding roots of continuous functions. It is based on the **Intermediate Value Theorem**, which guarantees that if a continuous function $f(x)$ changes sign over an interval $[a, b]$, then there exists at least one root $r$ such that $f(r) = 0$ within that interval.

## Theoretical Foundation

### The Intermediate Value Theorem

Let $f: [a, b] \to \mathbb{R}$ be a continuous function. If $f(a) \cdot f(b) < 0$, then there exists at least one $c \in (a, b)$ such that:

$$f(c) = 0$$

### Algorithm Description

The bisection method iteratively halves the interval $[a, b]$ and selects the subinterval where the sign change occurs:

1. Compute the midpoint: $c = \frac{a + b}{2}$

2. Evaluate $f(c)$

3. Determine the new interval:
   - If $f(a) \cdot f(c) < 0$, set $b = c$
   - Otherwise, set $a = c$

4. Repeat until $|b - a| < \epsilon$ or $|f(c)| < \delta$

### Convergence Analysis

After $n$ iterations, the interval width is:

$$|b_n - a_n| = \frac{b_0 - a_0}{2^n}$$

The error bound for the root approximation $c_n$ is:

$$|c_n - r| \leq \frac{b_0 - a_0}{2^{n+1}}$$

where $r$ is the true root. This gives us **linear convergence** with a guaranteed reduction of the error by half at each iteration.

### Number of Iterations Required

To achieve an accuracy of $\epsilon$, we need $n$ iterations where:

$$n \geq \frac{\ln(b_0 - a_0) - \ln(\epsilon)}{\ln 2} = \log_2\left(\frac{b_0 - a_0}{\epsilon}\right)$$

## Implementation

We will implement the bisection method and demonstrate it on the function:

$$f(x) = x^3 - x - 2$$

This function has a root near $x \approx 1.52$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bisection_method(f, a, b, tol=1e-10, max_iter=100):
    """
    Find a root of f(x) = 0 using the bisection method.
    
    Parameters:
    -----------
    f : callable
        The function for which we seek a root
    a, b : float
        Initial interval endpoints where f(a) and f(b) have opposite signs
    tol : float
        Tolerance for convergence
    max_iter : int
        Maximum number of iterations
        
    Returns:
    --------
    root : float
        Approximation of the root
    iterations : list
        History of (a, b, c, f(c)) for each iteration
    """
    # Verify initial conditions
    if f(a) * f(b) >= 0:
        raise ValueError("f(a) and f(b) must have opposite signs")
    
    iterations = []
    
    for i in range(max_iter):
        c = (a + b) / 2
        fc = f(c)
        
        iterations.append({
            'iteration': i + 1,
            'a': a,
            'b': b,
            'c': c,
            'f(c)': fc,
            'interval_width': b - a
        })
        
        # Check convergence
        if abs(fc) < tol or (b - a) / 2 < tol:
            return c, iterations
        
        # Update interval
        if f(a) * fc < 0:
            b = c
        else:
            a = c
    
    return c, iterations

## Example: Finding the Root of $f(x) = x^3 - x - 2$

In [ ]:
# Define the function
def f(x):
    return x**3 - x - 2

# Initial interval [1, 2]
a, b = 1.0, 2.0

# Verify sign change
print(f"f({a}) = {f(a):.4f}")
print(f"f({b}) = {f(b):.4f}")
print(f"Sign change: {f(a) * f(b) < 0}")

# Find the root
root, history = bisection_method(f, a, b, tol=1e-10)

print(f"\nRoot found: x = {root:.10f}")
print(f"f(root) = {f(root):.2e}")
print(f"Number of iterations: {len(history)}")

## Convergence History

In [ ]:
# Display iteration history
print("Iteration | a          | b          | c          | f(c)       | Width")
print("-" * 75)

for h in history[:15]:  # Show first 15 iterations
    print(f"{h['iteration']:^9} | {h['a']:.8f} | {h['b']:.8f} | {h['c']:.8f} | {h['f(c)']:+.2e} | {h['interval_width']:.2e}")

## Visualization

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Function and root location
ax1 = axes[0, 0]
x = np.linspace(0.5, 2.5, 500)
y = f(x)

ax1.plot(x, y, 'b-', linewidth=2, label=r'$f(x) = x^3 - x - 2$')
ax1.axhline(y=0, color='k', linewidth=0.5)
ax1.axvline(x=root, color='r', linestyle='--', alpha=0.7, label=f'Root: x = {root:.6f}')
ax1.plot(root, 0, 'ro', markersize=10)
ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel('f(x)', fontsize=12)
ax1.set_title('Function and Root Location', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Interval narrowing
ax2 = axes[0, 1]
iterations = [h['iteration'] for h in history]
a_vals = [h['a'] for h in history]
b_vals = [h['b'] for h in history]
c_vals = [h['c'] for h in history]

ax2.fill_between(iterations, a_vals, b_vals, alpha=0.3, color='blue', label='Interval [a, b]')
ax2.plot(iterations, c_vals, 'r.-', markersize=4, label='Midpoint c')
ax2.axhline(y=root, color='g', linestyle='--', alpha=0.7, label='True root')
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('x', fontsize=12)
ax2.set_title('Interval Convergence', fontsize=14)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Error convergence (semi-log)
ax3 = axes[1, 0]
errors = [abs(h['c'] - root) for h in history]
widths = [h['interval_width'] for h in history]

ax3.semilogy(iterations, errors, 'b.-', markersize=6, label='|c - root|')
ax3.semilogy(iterations, widths, 'r.-', markersize=6, label='Interval width')

# Theoretical bound
theoretical = [(b - a) / (2**(n+1)) for n in range(len(history))]
ax3.semilogy(iterations, theoretical, 'g--', linewidth=1.5, label='Theoretical bound')

ax3.set_xlabel('Iteration', fontsize=12)
ax3.set_ylabel('Error', fontsize=12)
ax3.set_title('Error Convergence (Semi-log Scale)', fontsize=14)
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Plot 4: |f(c)| convergence
ax4 = axes[1, 1]
fc_vals = [abs(h['f(c)']) for h in history]

ax4.semilogy(iterations, fc_vals, 'b.-', markersize=6)
ax4.set_xlabel('Iteration', fontsize=12)
ax4.set_ylabel('|f(c)|', fontsize=12)
ax4.set_title('Function Value Convergence', fontsize=14)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Convergence Rate Analysis

The bisection method exhibits **linear convergence**. We can verify this by examining the ratio of successive errors:

In [ ]:
# Analyze convergence rate
errors = [abs(h['c'] - root) for h in history]

print("Convergence Rate Analysis")
print("=" * 50)
print("\nRatio of successive errors (should be ~0.5 for linear convergence):")
print("\nIteration | Error        | Ratio")
print("-" * 40)

for i in range(1, min(len(errors), 15)):
    if errors[i-1] > 0:
        ratio = errors[i] / errors[i-1]
        print(f"{i+1:^9} | {errors[i]:.6e} | {ratio:.4f}")

# Verify theoretical prediction
expected_iterations = np.ceil(np.log2((2.0 - 1.0) / 1e-10))
print(f"\nTheoretical iterations needed for tol=1e-10: {expected_iterations:.0f}")
print(f"Actual iterations: {len(history)}")

## Comparison with Exact Solution

The polynomial $x^3 - x - 2 = 0$ can be factored as $(x - \alpha)(x^2 + \alpha x + \frac{2}{\alpha}) = 0$ where $\alpha$ is the real root. Using Cardano's formula or numerical verification:

In [ ]:
# Use numpy's polynomial root finder for comparison
coefficients = [1, 0, -1, -2]  # x^3 + 0*x^2 - x - 2
np_roots = np.roots(coefficients)

print("All roots of x³ - x - 2 = 0:")
for i, r in enumerate(np_roots):
    if np.isreal(r):
        print(f"  Root {i+1}: {r.real:.10f} (real)")
    else:
        print(f"  Root {i+1}: {r:.10f} (complex)")

real_root = np_roots[np.isreal(np_roots)][0].real
print(f"\nBisection result: {root:.10f}")
print(f"NumPy result:     {real_root:.10f}")
print(f"Difference:       {abs(root - real_root):.2e}")

## Summary

### Key Properties of the Bisection Method

**Advantages:**
- Guaranteed convergence for continuous functions with sign change
- Simple implementation
- Predictable number of iterations
- Robust against poor initial guesses (within the interval)

**Disadvantages:**
- Linear convergence (slower than Newton's method)
- Requires bracketing interval with sign change
- Cannot find roots of even multiplicity (no sign change)
- Only finds one root at a time

### Convergence Summary

For an initial interval $[a_0, b_0]$ and tolerance $\epsilon$:

$$n = \left\lceil \log_2\left(\frac{b_0 - a_0}{\epsilon}\right) \right\rceil$$

The bisection method is an excellent choice when robustness is more important than speed, or as a fallback when faster methods fail to converge.